In [1]:
library(forecast)
library(M4comp2018)
library(dplyr)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
etsl=load('Monthly_ets_datalist.RData')
ets_datalist <- eval(parse(text = etsl))
thetafl=load('Monthly_thetaf_datalist.RData')
thetaf_datalist <- eval(parse(text = thetafl))

In [3]:
ets_predh=ets_datalist[[2]]
thetaf_predh=thetaf_datalist[[2]]

In [4]:
ets_pred_res=ets_datalist[[1]]
thetaf_pred_res=thetaf_datalist[[1]]

In [5]:
etsoptl=load('Monthly_ets_opt_pre_result.RData')
ets_opt_pre_list<- eval(parse(text = etsoptl))
thetafoptl=load('Monthly_thetaf_opt_pre_result.RData')
thetaf_opt_pre_list<- eval(parse(text = thetafoptl))

In [6]:
time_matrix <- matrix(0,ncol = 3, nrow =3)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [7]:
ets_pr_min=ets_opt_pre_list[[1]]
ets_pr_mean=ets_opt_pre_list[[2]]
thetaf_pr_min=thetaf_opt_pre_list[[1]]
thetaf_pr_mean=thetaf_opt_pre_list[[2]]

In [8]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,3.537035,4,3.078049
4,3.015016,4,2.644279
4,3.787375,0,3.448685
3,2.275608,2,2.165220
0,1.425354,0,1.877339
4,2.767505,2,2.490559


In [9]:
ets_pr_min=ets_pr_min+1
ets_pr_mean=ets_pr_mean+1
thetaf_pr_min=thetaf_pr_min+1
thetaf_pr_mean=thetaf_pr_mean+1


In [10]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
5,4.537035,5,4.078049
5,4.015016,5,3.644279
5,4.787375,1,4.448685
4,3.275608,3,3.165220
1,2.425354,1,2.877339
5,3.767505,3,3.490559


In [11]:
round5=function(x)
    {
    l=round(x)
    for(i in 1:length(l))
    {
    if(l[i]>5)
        {
        l[i]=5
    }
    if(l[i]<1)
    {
        l[i]=1
    }
}
    return(l)
}

In [12]:
ets_pr_min[,2]=round5(ets_pr_min[,2])
ets_pr_min[,4]=round5(ets_pr_min[,4])

In [13]:
ets_pr_mean[,2]=round5(ets_pr_mean[,2])
ets_pr_mean[,4]=round5(ets_pr_mean[,4])

In [14]:
thetaf_pr_min[,2]=round5(thetaf_pr_min[,2])
thetaf_pr_min[,4]=round5(thetaf_pr_min[,4])

In [15]:
thetaf_pr_mean[,2]=round5(thetaf_pr_mean[,2])
thetaf_pr_mean[,4]=round5(thetaf_pr_mean[,4])

In [16]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
5,5,5,4
5,4,5,4
5,5,1,4
4,3,3,3
1,2,1,3
5,4,3,3


In [17]:
Monthly_M4 <- Filter(function(l) l$period == "Monthly", M4)

In [18]:
data=Monthly_M4

In [19]:
datalength=length(data)
h=data[[1]]$h
m=5
n=4
freq=frequency(data[[1]]$x)

In [20]:
realxx=matrix(0,datalength,h)
for(i in 1:datalength){
    realxx[i,]=data[[i]]$xx
}

In [21]:
length5=matrix(0,datalength,m)
for(k in 1:datalength)
{
y = data[[k]]$x
y_l=length(y)
loc = 1:length(y)
loc_m = as.integer(loc*m/length(y))
filt_d = data.frame(y,loc_m)
filt_0=filter(filt_d ,loc_m==0)
m_l=count(filt_0)
m_l=as.integer(m_l)
length5[k,]=c(0,1*m_l,2*m_l,3*m_l,4*m_l)
    }

In [22]:
end_time = Sys.time()
time_matrix[1,]=end_time-start_time

In [23]:
end_time-start_time

Time difference of 2.261149 mins

## ETS

In [24]:
start_time = Sys.time()

In [25]:
set.seed(100)
index = sample(2,length(data),replace = TRUE,prob=c(0.7,0.3))
testindex=c(1:length(data))[index==2]

In [26]:
meanpre=function(pred_array,opt_pr,h,testindex)
    {
    data_length=length(testindex)
    meanpreh=matrix(0,data_length,h)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    predh=pred_array[index,loc,,]
    predh=unique(predh)
    meanpreh[i,]=apply(predh,2,mean)
}
    return(meanpreh)
}

In [27]:
predres=function(pred_matrix,data,opt_pr,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]]$x)
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]]$x)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    start=lengthmatrix[i,loc]
    y_all=data[[index]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[index]]$xx
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [28]:
predres_mul=function(pred_matrix,data,opt_pr_mul,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]]$x)
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]]$x)
    for (i in 1:data_length){
    index=testindex[i]
    loc=min(opt_pr_mul[index,])
    start=lengthmatrix[i,loc]
    y_all=data[[index]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[index]]$xx
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [29]:
etsxgbcls_min=meanpre(ets_predh,ets_pr_min[,1],h,testindex)
etsxgbreg_min=meanpre(ets_predh,ets_pr_min[,2],h,testindex)
etslgbcls_min=meanpre(ets_predh,ets_pr_min[,3],h,testindex)
etslgbreg_min=meanpre(ets_predh,ets_pr_min[,4],h,testindex)

In [30]:
etsxgbcls_mean=meanpre(ets_predh,ets_pr_mean[,1],h,testindex)
etsxgbreg_mean=meanpre(ets_predh,ets_pr_mean[,2],h,testindex)
etslgbcls_mean=meanpre(ets_predh,ets_pr_mean[,3],h,testindex)
etslgbreg_mean=meanpre(ets_predh,ets_pr_mean[,4],h,testindex)

### The error of each improved prediction result

In [31]:
lengthtest=length5[testindex,]

In [32]:
etsminxgbclsres=predres(etsxgbcls_min,data,ets_pr_min[,1],lengthtest,h,testindex)
etsminxgbregres=predres(etsxgbreg_min,data,ets_pr_min[,2],lengthtest,h,testindex)
etsminlgbclsres=predres(etslgbcls_min,data,ets_pr_min[,3],lengthtest,h,testindex)
etsminlgbregres=predres(etslgbreg_min,data,ets_pr_min[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [33]:
etsmeanxgbclsres=predres(etsxgbcls_mean,data,ets_pr_mean[,1],lengthtest,h,testindex)
etsmeanxgbregres=predres(etsxgbreg_mean,data,ets_pr_mean[,2],lengthtest,h,testindex)
etsmeanlgbclsres=predres(etslgbcls_mean,data,ets_pr_mean[,3],lengthtest,h,testindex)
etsmeanlgbregres=predres(etslgbreg_mean,data,ets_pr_mean[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [34]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

In [35]:
end_time-start_time

Time difference of 5.052698 mins

### The results of the two methods of determining labels are averaged respectively

In [36]:
etsallmin=(etsxgbcls_min+etslgbcls_min+etsxgbreg_min+etslgbreg_min)/4
etsallmean=(etsxgbcls_mean+etslgbcls_mean+etsxgbreg_mean+etslgbreg_mean)/4

In [37]:
etsallminres=predres_mul(etsallmin,data,ets_pr_min,lengthtest,h,testindex)
etsallmeanres=predres_mul(etsallmean,data,ets_pr_mean,lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

### Under the above classification, the improved forecast results of the classification and regression models are averaged respectively

In [38]:
etsclsmin=(etsxgbcls_min+etslgbcls_min)/2
etsregmin=(etsxgbreg_min+etslgbreg_min)/2
etsclsmean=(etsxgbcls_mean+etslgbcls_mean)/2
etsregmean=(etsxgbreg_mean+etslgbreg_mean)/2

In [39]:
etsclsminres=predres_mul(etsclsmin,data,ets_pr_min[,c(1,3)],lengthtest,h,testindex)
etsregminres=predres_mul(etsregmin,data,ets_pr_min[,c(2,4)],lengthtest,h,testindex)
etsclsmeanres=predres_mul(etsclsmean,data,ets_pr_mean[,c(1,3)],lengthtest,h,testindex)
etsregmeanres=predres_mul(etsregmean,data,ets_pr_mean[,c(2,4)],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”


### Random slice

In [40]:
etspred_change<- aperm(ets_predh[1:datalength,1:m,1:n,1:h], c(1, 3, 2, 4))  
# 使用array函数将其转换为新的形状
etspred_change <- array(etspred_change, dim = c(datalength, m*n, h))

In [41]:
model=ets(data[[1]]$x)
fore_l=forecast(model,h=h)
etsrandomres=matrix(0,datalength,6)
for (i in 1:datalength){
    random=sample(m*n,n)
    minloc=min(random)
    # 使用cut函数将数据分割为区间
    loc <- cut(minloc, breaks =seq(0,m*n,n), labels = FALSE)
    loc<-as.numeric(loc)
    etsrandompred=apply(etspred_change[i,random,],2,mean)
    start=length5[i,loc]
    y_all=data[[i]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[i]]$xx
    fore_l$mean=ts(etsrandompred,start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    etsrandomres[i,]=res[2,1:6]
    }

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [42]:
mean1=function(x)
    {
# 使用is.finite函数检查哪些值是有限的
finite_values <- x[is.finite(x)]
# 计算有限值的平均值
d=mean(finite_values, na.rm = TRUE)
return(d)
}

In [43]:
print(apply(ets_pred_res[testindex,1,1,],2,mean1))
print(apply(etsminxgbclsres,2,mean1))
print(apply(etsminxgbregres,2,mean1))
print(apply(etsminlgbclsres,2,mean1))
print(apply(etsminlgbregres,2,mean1))

[1] -51.9639304 702.2183238 582.2703795  -7.6236245  16.8347235   0.9501302
[1] -50.8640541 694.6687708 576.5359710  -7.6301007  16.7047681   0.8717997
[1] -50.4280815 695.9867911 578.2768834  -7.5595263  16.7208829   0.8669765
[1] -53.764952 701.492960 582.807855  -7.845209  16.909792   1.001534
[1] -48.7513340 696.4124803 578.3232735  -7.5489926  16.7677653   0.8709729


In [44]:
print(apply(etsmeanxgbclsres,2,mean1))
print(apply(etsmeanxgbregres,2,mean1))
print(apply(etsmeanlgbclsres,2,mean1))
print(apply(etsmeanlgbregres,2,mean1))

[1] -48.3711147 687.8182951 569.8323829  -7.4693604  16.5158962   0.8335531
[1] -49.1664336 695.1922226 576.9591346  -7.5419041  16.6880220   0.8665721
[1] -53.156677 704.278045 584.675701  -7.861112  16.974309   1.005658
[1] -49.4986944 696.5021559 577.7833713  -7.4922626  16.6803929   0.8711882


In [45]:
print(apply(etsallminres,2,mean1))
print(apply(etsallmeanres,2,mean1))

[1] -50.9521054 685.5756087 569.2443843  -7.6459573  16.5292074   0.8799959
[1] -50.0482300 686.0339157 568.8815160  -7.5911598  16.4968031   0.8783802


In [46]:
print(apply(etsclsminres,2,mean1))
print(apply(etsregminres,2,mean1))
print(apply(etsclsmeanres,2,mean1))
print(apply(etsregmeanres,2,mean1))

[1] -52.3145030 687.4016464 570.6623481  -7.7376551  16.5711392   0.8885518
[1] -49.589708 694.275794 576.644128  -7.554259  16.704954   0.867736
[1] -50.7638960 686.9532811 569.4948492  -7.6652362  16.5416953   0.8762979
[1] -49.3325640 694.0142573 575.9059250  -7.5170833  16.6472096   0.8694831


In [47]:
print(apply(etsrandomres[testindex,],2,mean))

[1] -49.5712120 697.3417738 579.1670787  -7.8585253  16.8981799   0.9435307


In [48]:
etsall=matrix(0,16,6)
etsall[1,]=apply(ets_pred_res[testindex,1,1,],2,mean1)[1:6]
etsall[2,]=apply(etsminxgbclsres,2,mean1)
etsall[3,]=apply(etsminxgbregres,2,mean1)
etsall[4,]=apply(etsminlgbclsres,2,mean1)
etsall[5,]=apply(etsminlgbregres,2,mean1)
etsall[6,]=apply(etsmeanxgbclsres,2,mean1)
etsall[7,]=apply(etsmeanxgbregres,2,mean1)
etsall[8,]=apply(etsmeanlgbclsres,2,mean1)
etsall[9,]=apply(etsmeanlgbregres,2,mean1)
etsall[10,]=apply(etsallminres,2,mean1)
etsall[11,]=apply(etsallmeanres,2,mean1)
etsall[12,]=apply(etsclsminres,2,mean1)
etsall[13,]=apply(etsregminres,2,mean1)
etsall[14,]=apply(etsclsmeanres,2,mean1)
etsall[15,]=apply(etsregmeanres,2,mean1)
etsall[16,]=apply(etsrandomres[testindex,],2,mean)

In [49]:
etsall

-51.96393,702.2183,582.2704,-7.623625,16.83472,0.9501302
-50.86405,694.6688,576.5360,-7.630101,16.70477,0.8717997
-50.42808,695.9868,578.2769,-7.559526,16.72088,0.8669765
-53.76495,701.4930,582.8079,-7.845209,16.90979,1.0015339
-48.75133,696.4125,578.3233,-7.548993,16.76777,0.8709729
-48.37111,687.8183,569.8324,-7.469360,16.51590,0.8335531
-49.16643,695.1922,576.9591,-7.541904,16.68802,0.8665721
-53.15668,704.2780,584.6757,-7.861112,16.97431,1.0056579
-49.49869,696.5022,577.7834,-7.492263,16.68039,0.8711882
-50.95211,685.5756,569.2444,-7.645957,16.52921,0.8799959
-50.04823,686.0339,568.8815,-7.591160,16.49680,0.8783802


In [50]:
write.csv(etsall,'ets_Monthly_final_res.csv')

## thetaf

### The error of each improved prediction result

In [51]:
start_time = Sys.time()

In [52]:
thetafxgbcls_min=meanpre(thetaf_predh,thetaf_pr_min[,1],h,testindex)
thetafxgbreg_min=meanpre(thetaf_predh,thetaf_pr_min[,2],h,testindex)
thetaflgbcls_min=meanpre(thetaf_predh,thetaf_pr_min[,3],h,testindex)
thetaflgbreg_min=meanpre(thetaf_predh,thetaf_pr_min[,4],h,testindex)

In [53]:
thetafxgbcls_mean=meanpre(thetaf_predh,thetaf_pr_mean[,1],h,testindex)
thetafxgbreg_mean=meanpre(thetaf_predh,thetaf_pr_mean[,2],h,testindex)
thetaflgbcls_mean=meanpre(thetaf_predh,thetaf_pr_mean[,3],h,testindex)
thetaflgbreg_mean=meanpre(thetaf_predh,thetaf_pr_mean[,4],h,testindex)

In [54]:
thetafminxgbclsres=predres(thetafxgbcls_min,data,thetaf_pr_min[,1],lengthtest,h,testindex)
thetafminxgbregres=predres(thetafxgbreg_min,data,thetaf_pr_min[,2],lengthtest,h,testindex)
thetafminlgbclsres=predres(thetaflgbcls_min,data,thetaf_pr_min[,3],lengthtest,h,testindex)
thetafminlgbregres=predres(thetaflgbreg_min,data,thetaf_pr_min[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [55]:
thetafmeanxgbclsres=predres(thetafxgbcls_mean,data,thetaf_pr_mean[,1],lengthtest,h,testindex)
thetafmeanxgbregres=predres(thetafxgbreg_mean,data,thetaf_pr_mean[,2],lengthtest,h,testindex)
thetafmeanlgbclsres=predres(thetaflgbcls_mean,data,thetaf_pr_mean[,3],lengthtest,h,testindex)
thetafmeanlgbregres=predres(thetaflgbreg_mean,data,thetaf_pr_mean[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [56]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

In [57]:
end_time-start_time

Time difference of 5.07163 mins

### The results of the two methods of determining labels are averaged respectively

In [58]:
thetafallmin=(thetafxgbcls_min+thetaflgbcls_min+thetafxgbreg_min+thetaflgbreg_min)/4
thetafallmean=(thetafxgbcls_mean+thetaflgbcls_mean+thetafxgbreg_mean+thetaflgbreg_mean)/4

In [59]:
thetafallminres=predres_mul(thetafallmin,data,thetaf_pr_min,lengthtest,h,testindex)
thetafallmeanres=predres_mul(thetafallmean,data,thetaf_pr_mean,lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

### Under the above classification, the improved forecast results of the classification and regression models are averaged respectively

In [60]:
thetafclsmin=(thetafxgbcls_min+thetaflgbcls_min)/2
thetafregmin=(thetafxgbreg_min+thetaflgbreg_min)/2
thetafclsmean=(thetafxgbcls_mean+thetaflgbcls_mean)/2
thetafregmean=(thetafxgbreg_mean+thetaflgbreg_mean)/2

In [61]:
thetafclsminres=predres_mul(thetafclsmin,data,thetaf_pr_min[,c(1,3)],lengthtest,h,testindex)
thetafregminres=predres_mul(thetafregmin,data,thetaf_pr_min[,c(2,4)],lengthtest,h,testindex)
thetafclsmeanres=predres_mul(thetafclsmean,data,thetaf_pr_mean[,c(1,3)],lengthtest,h,testindex)
thetafregmeanres=predres_mul(thetafregmean,data,thetaf_pr_mean[,c(2,4)],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

### Random slice

In [62]:
thetafpred_change<- aperm(thetaf_predh[1:datalength,1:m,1:n,1:h], c(1, 3, 2, 4))  
# 使用array函数将其转换为新的形状
thetafpred_change <- array(thetafpred_change, dim = c(datalength, m*n, h))

In [63]:
model=ets(data[[1]]$x)
fore_l=forecast(model,h=h)
thetafrandomres=matrix(0,datalength,6)
for (i in 1:datalength){
    random=sample(m*n,n)
    minloc=min(random)
    # 使用cut函数将数据分割为区间
    loc <- cut(minloc, breaks =seq(0,m*n,n), labels = FALSE)
    loc<-as.numeric(loc)
    thetafrandompred=apply(thetafpred_change[i,random,],2,mean)
    start=length5[i,loc]
    y_all=data[[i]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[i]]$xx
    fore_l$mean=ts(thetafrandompred,start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    thetafrandomres[i,]=res[2,1:6]
    }

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [64]:
print(apply(thetaf_pred_res[testindex,1,1,],2,mean1))
print(apply(thetafminxgbclsres,2,mean1))
print(apply(thetafminxgbregres,2,mean1))
print(apply(thetafminlgbclsres,2,mean1))
print(apply(thetafminlgbregres,2,mean1))

[1]   3.0274860 679.0699856 563.0067910  -6.1592278  15.6812462   0.9728973
[1] -11.5390512 681.4025315 565.6777324  -5.9298913  15.7633691   0.8891093
[1]  -6.2628771 677.0750419 561.6987409  -6.0134954  15.7102159   0.8795226
[1] -24.169592 705.116226 587.441157  -6.966028  16.715838   1.037594
[1]  -5.6001802 676.6155282 561.2997816  -5.9861668  15.6826246   0.8830809


In [65]:
print(apply(thetafmeanxgbclsres,2,mean1))
print(apply(thetafmeanxgbregres,2,mean1))
print(apply(thetafmeanlgbclsres,2,mean1))
print(apply(thetafmeanlgbregres,2,mean1))

[1]  -7.103677 671.694979 556.935140  -5.799092  15.476079   0.842355
[1]  -3.4285646 673.5606980 558.5295930  -5.8895245  15.5756637   0.8742279
[1] -23.073267 704.812642 587.255104  -6.876428  16.713304   1.039613
[1]  -1.897820 674.316412 559.122577  -5.872343  15.585878   0.880468


In [66]:
print(apply(thetafallminres,2,mean1))
print(apply(thetafallmeanres,2,mean1))

[1] -11.8929251 674.2240066 559.4639368  -6.2238953  15.7212373   0.8942257
[1]  -8.875832 672.090742 557.673543  -6.109347  15.622072   0.892125


In [67]:
print(apply(thetafallminres,2,mean1))
print(apply(thetafallmeanres,2,mean1))

[1] -11.8929251 674.2240066 559.4639368  -6.2238953  15.7212373   0.8942257
[1]  -8.875832 672.090742 557.673543  -6.109347  15.622072   0.892125


In [68]:
print(apply(thetafclsminres,2,mean1))
print(apply(thetafclsmeanres,2,mean1))
print(apply(thetafregminres,2,mean1))
print(apply(thetafregmeanres,2,mean1))

[1] -17.8543215 682.3075965 566.9115333  -6.4479595  15.9903434   0.9071257
[1] -15.0884722 678.5025779 563.6766613  -6.3377601  15.8634950   0.8935573
[1]  -5.9315287 675.5369313 560.3355045  -5.9998311  15.6684355   0.8785633
[1]  -2.6631923 672.9390302 558.0029415  -5.8809340  15.5599314   0.8776865


In [69]:
print(apply(thetafrandomres[testindex,],2,mean1))

[1] -24.767375 693.471491 578.001257  -6.968365  16.426326   0.968620


In [70]:
thetafall=matrix(0,16,6)
thetafall[1,]=apply(thetaf_pred_res[testindex,1,1,],2,mean1)[1:6]
thetafall[2,]=apply(thetafminxgbclsres,2,mean1)
thetafall[3,]=apply(thetafminxgbregres,2,mean1)
thetafall[4,]=apply(thetafminlgbclsres,2,mean1)
thetafall[5,]=apply(thetafminlgbregres,2,mean1)
thetafall[6,]=apply(thetafmeanxgbclsres,2,mean1)
thetafall[7,]=apply(thetafmeanxgbregres,2,mean1)
thetafall[8,]=apply(thetafmeanlgbclsres,2,mean1)
thetafall[9,]=apply(thetafmeanlgbregres,2,mean1)
thetafall[10,]=apply(thetafallminres,2,mean1)
thetafall[11,]=apply(thetafallmeanres,2,mean1)
thetafall[12,]=apply(thetafclsminres,2,mean1)
thetafall[13,]=apply(thetafclsmeanres,2,mean1)
thetafall[14,]=apply(thetafregminres,2,mean1)
thetafall[15,]=apply(thetafregmeanres,2,mean1)
thetafall[16,]=apply(thetafrandomres[testindex,],2,mean1)

In [71]:
thetafall

3.027486,679.0700,563.0068,-6.159228,15.68125,0.9728973
-11.539051,681.4025,565.6777,-5.929891,15.76337,0.8891093
-6.262877,677.0750,561.6987,-6.013495,15.71022,0.8795226
-24.169592,705.1162,587.4412,-6.966028,16.71584,1.0375938
-5.600180,676.6155,561.2998,-5.986167,15.68262,0.8830809
-7.103677,671.6950,556.9351,-5.799092,15.47608,0.8423550
-3.428565,673.5607,558.5296,-5.889525,15.57566,0.8742279
-23.073267,704.8126,587.2551,-6.876428,16.71330,1.0396133
-1.897820,674.3164,559.1226,-5.872343,15.58588,0.8804680
-11.892925,674.2240,559.4639,-6.223895,15.72124,0.8942257
-8.875832,672.0907,557.6735,-6.109347,15.62207,0.8921250


In [72]:
write.csv(thetafall,'thetaf_Monthly_final_res.csv')

In [73]:
time_matrix

user_time,system_time,elapsed_time
2.261149,2.261149,2.261149
5.052698,5.052698,5.052698
5.071630,5.071630,5.071630


In [74]:
write.csv(time_matrix,'Monthly_time_matrix.csv')